In [7]:
import pandas as pd

countries = pd.read_csv("Countries1.csv")
country_codes = countries["Alpha2"].tolist() # Get country code column

['AF', 'AL', 'DZ', 'AS', 'AD', 'AO', 'AI', 'AG', 'AR', 'AM', 'AW', 'AU', 'AT', 'AZ', 'BS', 'BH', 'BD', 'BB', 'BY', 'BE', 'BZ', 'BJ', 'BM', 'BT', 'BO', 'BQ', 'BA', 'BW', 'BR', 'VG', 'BN', 'BG', 'BF', 'BI', 'CV', 'KH', 'CM', 'CA', 'KY', 'CF', 'TD', 'JG', 'CL', 'CN', 'HK', 'MO', 'CO', 'KM', 'CG', 'CK', 'CR', 'CI', 'HR', 'CU', 'CW', 'CY', 'CZ', 'KP', 'CD', 'DK', 'DJ', 'DM', 'DO', 'EC', 'EG', 'SV', 'GQ', 'ER', 'EE', 'SZ', 'ET', 'FK', 'FO', 'FJ', 'FI', 'FR', 'GF', 'PF', 'GA', 'GM', 'GE', 'DE', 'GH', 'GI', 'GR', 'GL', 'GD', 'GP', 'GU', 'GT', 'GN', 'GW', 'GY', 'HT', 'VA', 'HN', 'HU', 'IS', 'IN', 'ID', 'IR', 'IQ', 'IE', 'IM', 'IL', 'IT', 'JM', 'JP', 'JO', 'KZ', 'KE', 'KI', 'KW', 'KG', 'LA', 'LV', 'LB', 'LS', 'LR', 'LY', 'LI', 'LT', 'LU', 'MG', 'MW', 'MY', 'MV', 'ML', 'MT', 'MH', 'MQ', 'MR', 'MU', 'YT', 'MX', 'FM', 'MC', 'MN', 'ME', 'MS', 'MA', 'MZ', 'MM', nan, 'NR', 'NP', 'NL', 'NC', 'NZ', 'NI', 'NE', 'NG', 'NU', 'MK', 'MP', 'NO', 'OM', 'PK', 'PW', 'PA', 'PG', 'PY', 'PE', 'PH', 'PL', 'PT', 'PR'

In [13]:
import requests
from pathlib import Path

session = requests.Session()

base_url = "https://data.countrydata.iatistandard.org/output/web/xlsx/en"
out_dir = Path("data/country_xlsx")
out_dir.mkdir(parents=True, exist_ok=True)

failed = []

for code in country_codes:
    url = f"{base_url}/{code}.xlsx"
    out_path = out_dir / f"{code}.xlsx"

    if out_path.exists():
        continue

    try:
        r = session.get(url, timeout=60)
        r.raise_for_status()

        with open(out_path, "wb") as f:
            f.write(r.content)

    except requests.exceptions.HTTPError:
        failed.append(code)

print("Done.")
print("Failed codes:", failed)


Done.
Failed codes: ['JG', nan]
